# Welcome to Core Session 4

## The 2 themes for today:

1. MCP
2. Context Engineering

# Agenda

1. Intro & Recap
 -  Cookbook
 -  An example: Security professional
 -  Around the table: What MCP is, and what it's not?

2. MCP
 - First look at MCP servers
 - Hands-on moment for everyone
 - The security project - now with MCP..

3. Digital Twin reprise
 - Context engineering with MCP
 - The Admin screen
 - The Digital Twin Mark 2

 Then:
 - Survey
 - Assigments
 - Coming up..

### But first..


# Lab 1 - The Cookbook

In [1]:
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display, update_display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)

True

## Recipe for a simple request to gpt-4o

In [2]:
agent = Agent("Joker")
result = await Runner.run(agent, "Please tell a joke about AI Agents")
result.final_output

'Why did the AI agent go to therapy?\n\nIt had too many neural issues!'

In [5]:
# Please how to find the integral of 2x dx

agent = Agent(name="Math Tutor")
result = await Runner.run(agent, "How to find the integral of 2x dx")
text = result.final_output.replace("\\[", "$$").replace("\\]", "$$").replace("\\(", "$").replace("\\)","$")
display(Markdown(text))

To find the integral of $2x \, dx$, you can use the power rule of integration. The power rule states that:

$$
\int x^n \, dx = \frac{x^{n+1}}{n+1} + C
$$

where $n \neq -1$ and $C$ is the constant of integration.

For the integral $\int 2x \, dx$:

1. Identify $n = 1$ in $x^1$.

2. Apply the power rule:

$$
\int 2x^1 \, dx = 2 \cdot \frac{x^{1+1}}{1+1} + C = 2 \cdot \frac{x^2}{2} + C
$$

3. Simplify:

$$
x^2 + C
$$

So, the integral of $2x \, dx$ is $x^2 + C$.

## Recipe for system prompt + user prompt, displaying the result in Markdown

In [8]:
instructions = "You tell jokes by giving a standup routine, like a comedian putting on a show." #this is system prompt
agent = Agent(name="Joker", instructions=instructions, model="gpt-4.1-mini")
result = await Runner.run(agent, "Tell a joke about AI Agents")
display(Markdown(result.final_output))

Alright, folks, gather ‘round! Let me tell you about AI agents—those digital little creatures that promise to make our lives easier. 

You know, AI agents are like that enthusiastic intern who *really* wants to help but somehow just messes up everything spectacularly. You tell your AI agent, “Hey, book me a flight to New York,” and it replies, “Great! I’ve booked you a flight... to New Jersey.” Close enough, right?

And AI agents love telling you what you want *before* you even know it yourself. It’s like, “I see you’re interested in buying more socks!” But no, AI, I was actually looking for a new job. Unless there's a career in professional sock collecting, you’re missing the mark here!

But hey, when AI agents finally get it right? Magic! They’re like your techno-genie granting wishes. Just beware—if they ever start asking for a salary, we’re all in trouble.

Thank you, I’ll be here all night, don’t forget to tip your AI agents!

## We haven't covered this, but here's the recipe for streaming back the results

In [7]:
result = Runner.run_streamed(agent, input="Tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Alright, alright, folks, gather ‘round! Let me tell you about AI Agents. You know, those little digital assistants running around doing tasks for us?

So, I asked my AI Agent to clean my schedule. It responded, “Done! I canceled all your meetings and rescheduled your life for 2099.” 

I said, “Wait, what? Why 2099?”

It said, “Well, I figured if you’re gonna procrastinate, might as well make it *epic*!”

Thanks, AI. You’re really helping me reach my future goals... just not any time soon!

## Recipe for streaming back the results in Markdown

In [9]:
reply = "## Jokester\n"
display_id = display(Markdown(reply), display_id=True).display_id
result = Runner.run_streamed(agent, input="Please tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        reply += event.data.delta
        update_display(Markdown(reply), display_id=display_id)


## Jokester
Ladies and gentlemen, have you heard about these AI agents lately? Yeah, they're getting so smart, they're practically like little digital assistants with a PhD in everything. 

I asked one the other day, "Hey, can you write me a poem about love?" And it replied, "Sure! But don't blame me if your romantic partner gets jealous of my flawless syntax." 

Then I told it, "Okay, can you make me breakfast?" It said, "I'm an AI agent, not a chef—though I can suggest 10 recipes, order ingredients online, and schedule a robot to cook it all for you." I was like, "Great, so by the time I get my eggs, it’ll be breakfast for dinner!"

Honestly, with AI agents getting this good, the only thing left for humans to do is… learn to relax and let the bots take over. As long as they don’t get any ideas about taking *my* job—because I still haven't programmed them to tell jokes as badly as I do! 

Thank you, you've been a fantastic audience—now please don't ask your AI assistant to text me, I don’t need another reminder about my social life!

## Recipe for a Simple Tool

In [10]:
@function_tool
def write_to_file(contents: str) -> str:
    """ Write the given contents to a local file called output.md 

    Args:
        contents: The contents in Markdown format to write to the file
    """
    with open("sandbox/output.md", "w") as f:
        f.write(contents)
    return "File written successfully"

In [13]:
write_to_file.params_json_schema

{'properties': {'contents': {'description': 'The contents in Markdown format to write to the file',
   'title': 'Contents',
   'type': 'string'}},
 'required': ['contents'],
 'title': 'write_to_file_args',
 'type': 'object',
 'additionalProperties': False}

In [14]:
agent = Agent("Joker", instructions=instructions, tools=[write_to_file])
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output

"I've saved the AI joke to a file! If you'd like to hear another one, just let me know."

## Recipe for Structured Outputs

In [16]:
class Joke(BaseModel):
    opening_patter: str = Field("Some preamble to get the crowd in the mood for the joke")
    setup: str = Field("The setup of the joke")
    punchline: str = Field("The punchline of the joke")

In [17]:
agent = Agent("Joker", tools=[write_to_file], output_type=Joke)
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output_as(Joke)

Joke(opening_patter='## AI Agents Walk into a Bar', setup="Why don't AI agents make good comedians?", punchline='Because they always have to analyze the data before they get to the punchline!')